In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers

In [ ]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [ ]:
from openpyxl import Workbook
from openpyxl import load_workbook

In [ ]:
import pathlib
from datetime import datetime

In [ ]:
import itertools
import os

In [ ]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "colorectal_histology",
    split=["train[:85%]", "train[85%:95%]", "train[95%:]"],
    with_info=True,
    as_supervised=True,
    shuffle_files= True,
)

num_classes = info.features['label'].num_classes

In [ ]:
size = (150, 150)
batch_size = 16

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

In [ ]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1)

In [ ]:
%cd  functions

In [3]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from Simulation import save_obj, load_obj

In [ ]:
model = GetNeuralNetworkModel('VGG19',(150,150,3),8, quantization = False, aging_active=False)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
%cd  ../VGG19/Weights/

In [ ]:
model.load_weights('weights.data')

In [ ]:
score = model.evaluate(test_ds, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
for index,layer in enumerate(model.layers):
    print(index,layer.name)

In [ ]:
num_address  = 2359808
LI = [1,3,5,8,10,12,15,17,19,21,23,26,28,30,32,34,37,39,41,43,45,49,53]

AI = [2,4,7,9,11,14,16,18,20,22,25,27,29,31,33,36,38,40,42,44,47,52,56]

Indices      = [1,3,5,8,10,12,15,17,19,21,23,26,28,30,32,34,37,39,41,43,45,49,53,57]
# lamdas =[]

samples      = 150

Data         = GetReadAndWrites(model,Indices,num_address,samples,CNN_gating=False)
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)

In [ ]:
df = QuantizationEffect('VGG19',test_ds,'weights.data',(150,150,3),8,batch_size)

In [ ]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/byte_x/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../VGG19/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('VGG19', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 9, act_int_size = 8, wgt_frac_size = 11, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    print(loss)    
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('VGG19_fichero_fichero_Flip-patch.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

In [ ]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/ECC/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../VGG19/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('VGG19', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 9, act_int_size = 8, wgt_frac_size = 11, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    print(loss)    
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('VGG19_fichero_fichero_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

In [ ]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/ECCx/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../VGG19/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('VGG19', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 9, act_int_size = 8, wgt_frac_size = 11, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    print(loss)    
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('VGG19_fichero_fichero_Iso-Area_ECC.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

In [ ]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/mask_volteada/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../VGG19/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('VGG19', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 9, act_int_size = 8, wgt_frac_size = 11, wgt_int_size = 12,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    print(loss)    
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('VGG19_fichero_fichero_mask_volteada.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))

In [ ]:
Accs = []
Loss = []
Volt = []

vol=54
for i in range(7):
    %cd '../../Data/base_line/VC_707'
    error_mask = load_obj('error_mask_0' + str(vol))
    locs = load_obj('locs_0' + str(vol))

    vol = vol + 1
    print('tamaño de locs', len(locs))
    
    %cd  '../../../VGG19/Weights/'
    
    loss,acc   = CheckAccuracyAndLoss('VGG19', test_ds, 'weights.data', output_shape=8, input_shape = (150,150,3),
                                            act_frac_size = 8, act_int_size = 8, wgt_frac_size = 14, wgt_int_size = 0,
                                            batch_size=batch_size, verbose = 1, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    Accs.append(acc)
    Loss.append(loss)
    Volt.append(vol-1)    
    print(acc)
    print(loss)    
    
Acc_Net=pd.DataFrame(Accs)
Loss_Net=pd.DataFrame(Loss)
Volt_Net=pd.DataFrame(Volt)

buf_cero = pd.concat([Acc_Net, Loss_Net, Volt_Net], axis=1, join='outer')
buf_cero.columns =['Acc','Loss','Volt']
buf_cero.to_excel('VGG19_fichero_fichero_base_line.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))